## SET UP

In [49]:
#importing necessary libraries
import re
import ast
import time
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain.chains import LLMChain
from langchain_openai import AzureChatOpenAI

In [50]:
openai_api_key = "EJgyDSQSrrt8FyS9qCoXWExkJsALhSg77kWL9qt6L592RYCyGIw4JQQJ99BBAC77bzfXJ3w3AAABACOG01wO"
azure_endpoint = r"https://ai-pocs.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2024-08-01-preview"

In [51]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4",  # or your deployment
    api_version="2024-08-01-preview",  # or your api version
    api_key=openai_api_key,  
    azure_endpoint = azure_endpoint
)

In [52]:
# Connect to the database (e.g., SQLite)
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

## Table Infos & Structures

In [5]:
# Query to get all table names
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query and fetch results
table_names = db.run(query)

print(table_names)

[('sqlite_sequence',), ('Customers',), ('Accounts',), ('Transactions',), ('PaymentsDue',), ('Invoices',)]


In [6]:
#Customers table info
query="PRAGMA table_info(Customers);"

print(db.run(query))

[(0, 'customer_id', 'INTEGER', 0, None, 1), (1, 'first_name', 'TEXT', 0, None, 0), (2, 'last_name', 'TEXT', 0, None, 0), (3, 'email', 'TEXT', 0, None, 0), (4, 'phone_number', 'TEXT', 0, None, 0), (5, 'address', 'TEXT', 0, None, 0)]


In [7]:
# Query to view the structure of the Customers table
query = "Select * from Customers;"

# Execute the query to get table structure
table_structure = db.run(query)

# Display the table structure
print(table_structure)

[(1, 'John', 'Doe', 'john.doe@email.com', '123-456-7890', '123 Main St, Cityville'), (2, 'Jane', 'Smith', 'jane.smith@email.com', '234-567-8901', '456 Elm St, Townsville'), (3, 'Mike', 'Johnson', 'mike.johnson@email.com', '345-678-9012', '789 Oak St, Villageburg')]


In [8]:
#Accounts Table Info
query="PRAGMA table_info(Accounts);"

print(db.run(query))

[(0, 'account_id', 'INTEGER', 0, None, 1), (1, 'customer_id', 'INTEGER', 0, None, 0), (2, 'account_type', 'TEXT', 0, None, 0), (3, 'balance', 'REAL', 0, None, 0)]


In [9]:
# Query to view the structure of the Accounts table
query = "Select * from Accounts;"

# Execute the query to get table structure
table_structure = db.run(query)

# Display the table structure
print(table_structure)

[(1, 1, 'Checking', 5000.75), (2, 1, 'Credit', -250.0), (3, 2, 'Savings', 3000.0), (4, 2, 'Checking', 1200.5), (5, 3, 'Savings', 15000.25)]


In [10]:
#Transactions table info
query="PRAGMA table_info(Transactions);"

print(db.run(query))

[(0, 'transaction_id', 'INTEGER', 0, None, 1), (1, 'account_id', 'INTEGER', 0, None, 0), (2, 'transaction_type', 'TEXT', 0, None, 0), (3, 'amount', 'REAL', 0, None, 0), (4, 'transaction_date', 'TEXT', 0, None, 0), (5, 'description', 'TEXT', 0, None, 0)]


In [11]:
# Query to view the structure of the Transactions table
query = "Select * from Transactions;"

# Execute the query to get table structure
table_structure_transactions = db.run(query)

# Display the table structure
print(table_structure_transactions)

[(1, 1, 'Deposit', 1500.0, '2025-02-10', 'Direct deposit from employer'), (2, 1, 'Payment', 200.0, '2025-02-12', 'Payment for utilities'), (3, 2, 'Withdrawal', 100.0, '2025-02-15', 'ATM Withdrawal'), (4, 3, 'Deposit', 2000.0, '2025-02-16', 'Wire transfer'), (5, 4, 'Payment', 50.0, '2025-02-18', 'Credit card payment')]


In [12]:
#PaymentsDue table info
query="PRAGMA table_info(PaymentsDue);"

print(db.run(query))

[(0, 'payment_id', 'INTEGER', 0, None, 1), (1, 'account_id', 'INTEGER', 0, None, 0), (2, 'payment_amount', 'REAL', 0, None, 0), (3, 'due_date', 'TEXT', 0, None, 0), (4, 'payment_status', 'TEXT', 0, None, 0)]


In [13]:
# Query to view the structure of the PaymentsDue table
query = "Select * from PaymentsDue;"

# Execute the query to get table structure
table_structure_paymentsdue = db.run(query)

# Display the table structure
print(table_structure_paymentsdue)

[(1, 1, 300.0, '2025-02-20', 'Due'), (2, 1, 50.0, '2025-02-25', 'Paid'), (3, 2, 500.0, '2025-02-22', 'Late'), (4, 3, 150.0, '2025-02-23', 'Due'), (5, 4, 75.0, '2025-02-28', 'Due')]


## Table & It's Column Descriptions

In [54]:
table_column_descriptions={"Customers":{"customer_id":"Each Customer has different ids",
                                        "first_name":"First name of the customer",
                                        "last_name":"Last Name of the customer",
                                        "email":"email address of the customer",
                                        "phone_number":"phone number of the customer"},
                           "Accounts":{"account_id":"Account id of different accounts. Each customer can has multiple accounts",
                                       "customer_id":"Customer Id of the customer which has the account.",
                                       "account_type":"The type of the account Savings, credit etc",
                                       "balance":"balance amount in the account"},
                           "Transactions":{"transaction_id":"Each transaction has different transaction ids. Each account can do multiple transactions",
                                           "account_id":"Account ID of the account which made the transaction.",
                                           "transaction_type": "The type of the transaction like payment, withdrawal, deposit etc",
                                           "amount":"amount money of the transaction",
                                           "transaction_date":"The date in which the transaction occured.It's in YYYY-MM-DD format.",
                                           "description":"The description of the transaction on how or for which the transaction's being done"
                                          },
                            "PaymentsDue":{"payment_id":"Every due payment has a payment id",
                                           "account_id":"The account id of the customer which has the payment due",
                                           "payment_amount":"The money amount that is due for payment",
                                           "due_date":"The due date before which the customer has to make the payment. It's in YYYY-MM-DD format",
                                           "payment_status":"The status of the payment like due, paid, late etc"
                                          }
                          }

## Schema Information

In [55]:
# Query to get all table names
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query and fetch results
table_names = db.run(query)

#Changing the string output to it's literal form.
table_names=ast.literal_eval(table_names)

#list of table names.
table_names=[item[0] for item in table_names]

#table, column & type pair
all_tab_col=""
for table_name in table_names:
    query = f"PRAGMA table_info({table_name});"
    result = db.run(query)
    result=ast.literal_eval(result)
    
    if isinstance(result, list) and len(result) > 0:
        # Extract the second element (column name) from each tuple if it's valid
        cols= [str({row[1]:row[2]}) for row in result]
    tab_col=f"{table_name} ({','.join(cols)}) "
    all_tab_col=all_tab_col+'\n'+tab_col
    
print(all_tab_col)


sqlite_sequence ({'name': ''},{'seq': ''}) 
Customers ({'customer_id': 'INTEGER'},{'first_name': 'TEXT'},{'last_name': 'TEXT'},{'email': 'TEXT'},{'phone_number': 'TEXT'},{'address': 'TEXT'}) 
Accounts ({'account_id': 'INTEGER'},{'customer_id': 'INTEGER'},{'account_type': 'TEXT'},{'balance': 'REAL'}) 
Transactions ({'transaction_id': 'INTEGER'},{'account_id': 'INTEGER'},{'transaction_type': 'TEXT'},{'amount': 'REAL'},{'transaction_date': 'TEXT'},{'description': 'TEXT'}) 
PaymentsDue ({'payment_id': 'INTEGER'},{'account_id': 'INTEGER'},{'payment_amount': 'REAL'},{'due_date': 'TEXT'},{'payment_status': 'TEXT'}) 
Invoices ({'InvoiceId': 'INTEGER'},{'AccountId': 'INTEGER'},{'InvoiceDate': 'TEXT'},{'DueDate': 'TEXT'},{'Total': 'REAL'}) 


## Chains & Tools

In [24]:
#the template will give a sqllite sql query & the tables used in the sql query as an output.
template="""
You are a SQL expert. Based on the following database schema, generate a SQL query to answer the user's question. You can use multiple tables. Also give the relevant tables that's being used in the SQL query.

Database Schema:
{all_tab_col}

table column details:
{table_column_descriptions}

User Question: {query}

Please give the output in the following JSON format:
SQL Query: ONLY give the sql query generated as an output. I want to directly feed it into the sql query.,
Relevant Tables: The column used in the SQL Query. Just the Table Names in comma separated format.

"""

#prompttemplate takes table,column group, query, column descriptions of all tables.
query_generation_prompt = PromptTemplate(
    input_variables=["query", "all_tab_col","table_column_descriptions"],
    template=template
)

query_generation_chain = LLMChain(prompt=query_generation_prompt, llm=llm)

In [25]:
#query generation tool & it's function
def query_generation(query, all_tab_col, table_column_descriptions):
    return query_generation_chain.run(query=query, all_tab_col=all_tab_col, table_column_descriptions=table_column_descriptions)

query_generation_tool=Tool(
        name="Query Generation",
        func=query_generation,
        description="Query Generation"
    )

In [38]:
#the prompt template will give the final response of the query in natural language
template="""I will give you a query & it's answer which is extracted through a sql query along with the table's info from which it's extracted from & the descriptions of each column of the table. You will have to answer the query in English.

Tips:
**Please take into account all the informations of all the tables. Also each column of each table has different functions.**
Think thrice before giving an answer.
Summarize the answer in 150 tokens. Make the answer clear & concise with exact numbers & do not create numbers by yourself.

Steps:
1.Understand the query.
2.Understand the SQL query: columns, tables used & how it's being used. 
3.Check the table information of each table: which columns being used in the sql query & what's the sql query asking for.
4.Understand the response of the sql query which each value represent.

Query:{query},

Answer:{result},

SQL Query:{sql_query},

Table Column Descriptions: {table_column_details}
"""

NL_response_prompt=PromptTemplate(
    input_variables=["query","result","sql_query","table_column_details"],
    template=template
)

NL_response_chain=LLMChain(prompt=NL_response_prompt,llm=llm)

In [39]:
def NL_response(query,result,sql_query,table_column_details):
    return NL_response_chain.run(query=query,result=result,sql_query=sql_query,table_column_details=table_column_details)

NL_response_tool=Tool(
    name="NL_response",
    func=NL_response,
    description="Natural Language Response"
)

## Main Function

In [47]:
def final_response(query,table_column_descriptions,all_tab_col):
    
    sql_query_table=query_generation_tool.func(query, all_tab_col, table_column_descriptions)
    print(sql_query_table)#debugging
    
    sql_query_table=ast.literal_eval(sql_query_table)
    sql_query=sql_query_table["SQL Query"]
    
    print(sql_query)
        
    result=db.run(sql_query)
    
    print(result)#debugging
    
    selected_tables=sql_query_table["Relevant Tables"].split(',')
    selected_tables=[selected_table.strip() for selected_table in selected_tables]
    table_column_details=""
    for selected_table in selected_tables:
        table_column_detail=table_column_descriptions[f"{selected_table}"]
        table_column_details=table_column_details+f"\n{selected_table} : {str(table_column_detail)}"
    
    print(table_column_details)#debugging
    
    response=NL_response_tool.func(query,result,sql_query,table_column_details)
    
    return response   

## Response of Different Queries

In [35]:
start_time=time.time()
query = "What is the payment due date for account 1?"
response=final_response(query,table_column_descriptions,all_tab_col)
end_time=time.time()
execution_time=end_time-start_time
print("Execution Times in seconds:",execution_time)
print("Response:\n",response)

Execution Times in seconds: 3.6209194660186768
Response:
 The payment due dates for account number 1 are on February 20, 2025, and February 25, 2025.


In [56]:
start_time=time.time()
query="Give me account id 1 customer's transaction details in 12th February 2025?"
response=final_response(query,table_column_descriptions,all_tab_col)
end_time=time.time()
execution_time=end_time-start_time
print("Execution Times in seconds:",execution_time)
print("Response:\n",response)

{
  "SQL Query": "SELECT * FROM Transactions WHERE account_id = 1 AND transaction_date = '2025-02-12'",
  "Relevant Tables": "Transactions"
}
SELECT * FROM Transactions WHERE account_id = 1 AND transaction_date = '2025-02-12'
[(2, 1, 'Payment', 200.0, '2025-02-12', 'Payment for utilities')]

Transactions : {'transaction_id': 'Each transaction has different transaction ids. Each account can do multiple transactions', 'account_id': 'Account ID of the account which made the transaction.', 'transaction_type': 'The type of the transaction like payment, withdrawal, deposit etc', 'amount': 'amount money of the transaction', 'transaction_date': "The date in which the transaction occured.It's in YYYY-MM-DD format.", 'description': "The description of the transaction on how or for which the transaction's being done"}
Execution Times in seconds: 5.283830642700195
Response:
 On February 12, 2025, the customer with account ID 1 made a single transaction. This transaction had an ID of 2, was a payme

In [34]:
start_time=time.time()
query = "Give me customer id 1 customer's all transaction details?"
response=final_response(query,table_column_descriptions,all_tab_col)
end_time=time.time()
execution_time=end_time-start_time
print("Execution Times in seconds:",execution_time)
print("Response:\n",response)

Execution Times in seconds: 7.574652910232544
Response:
 Customer ID 1 has made three transactions across two accounts. The first transaction is a deposit of $1500 on February 10, 2025, into their Checking account, described as a direct deposit from their employer, leaving the account with a balance of $5000.75. The second is a payment of $200 on February 12, 2025, for utilities, also from the Checking account, maintaining the same balance of $5000.75. The third transaction is a withdrawal of $100 on February 15, 2025, from their Credit account through an ATM, leaving the balance at -$250.
